In [48]:
# importing all the required modules
import PyPDF2
import itertools
import pandas as pd

In [2]:
# creating an object 
file = open('u63.pdf', 'rb')

# creating a pdf reader object
fileReader = PyPDF2.PdfFileReader(file)

# print the number of pages in pdf file
print(fileReader.numPages)

86


In [4]:
def extractTextList(self):
    text_list = []
    content = self["/Contents"].getObject()
    if not isinstance(content, ContentStream):
        content = ContentStream(content, self.pdf)

    for operands, operator in content.operations:
        if operator == b_("Tj"):
            _text = operands[0]
            if isinstance(_text, TextStringObject) and len(_text.strip()):
                text_list.append(_text.strip())
        elif operator == b_("T*"):
            pass
        elif operator == b_("'"):
            pass
            _text = operands[0]
            if isinstance(_text, TextStringObject) and len(operands[0]):
                text_list.append(operands[0])
        elif operator == b_('"'):
            _text = operands[2]
            if isinstance(_text, TextStringObject) and len(_text):
                text_list.append(_text)
        elif operator == b_("TJ"):
            for i in operands[0]:
                if isinstance(i, TextStringObject) and len(i):
                    text_list.append(i)
    return text_list

In [76]:
from PyPDF2.pdf import PageObject, u_, ContentStream, b_, TextStringObject
PageObject.extractTextList = extractTextList

In [77]:
def between(text_elements, drop_while, take_while):    
    return list(itertools.takewhile(take_while, itertools.dropwhile(drop_while, text_elements)))[1:]

In [78]:
pdfFileObj = open('u63.pdf', 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

page0 = pdfReader.getPage(0)
text_elements = page0.extractTextList()

lines = between(text_elements, lambda x: x != 'Name', lambda x: 'Category' not in x)
print('\n'.join(lines))

C6351B


In [79]:
output = []
for i in range(pdfReader.numPages):
    page0 = pdfReader.getPage(i)
    text_elements = page0.extractTextList()
    output += text_elements
len(output)

8952

In [80]:
def split_val(lisst ,value, equipements = False):
    size = len(lisst)
    if not equipements:
        idx_list = [idx for idx, val in enumerate(lisst) if val == value]
        res = [lisst[i: j] for i, j in zip([0] + idx_list, idx_list + ([size] if idx_list[-1] != size else []))]
    else:
        idx_list = [idx for idx, val in enumerate(lisst) if value in val]
        if idx_list == []:
            res = lisst
        else:
            res = [lisst[i: j] for i, j in zip([0] + idx_list, idx_list + ([size] if idx_list[-1] != size else []))]
    return res

In [82]:
res = split_val(output ,'Name')
column= ['Type inspection', 'Nom equipement', 'Date inspection']
Data = pd.DataFrame(columns= column )

for i in range(len(res)):
    ti = between(res[i], lambda x: x != 'Name', lambda x: 'Category' not in x)
    if ti == []:
        continue
    res_event = split_val(res[i] ,'EVENTMASTER',equipements=True)
    for j in range(len(res_event)):
        ne_di = between(res_event[j], lambda x: 'EVENTMASTER' not in x , lambda x: 'Parent' not in x)
        if ne_di != []:
            s = pd.Series([ti[0],ne_di[0],ne_di[1]],index=column)
            Data = Data.append(s,ignore_index=True)
        ne_di = []
    ti = []

Data[:10]


,Type inspection,Nom equipement,Date inspection
0,C6351B,RBI,31 dec 2019
1,C6351B,HIST,27 dec 2016
2,C6351B,NVISI,20 apr 2010
3,C6351B,XVISI,21 mei 2005
4,C6351B,HIST,01 mei 2000
5,E6301,RBI,31 dec 2019
6,E6301,HIST,20 mei 2018
7,E6301,HIST,01 jul 2015
8,E6301,HIST,21 apr 2015
9,E6301,HIST,08 apr 2015


In [83]:
len(Data)

287

In [85]:
Data.to_csv('Data_extracted_from_pdf.csv', index=False)